In [1]:
!pip install -q recommenders
!pip install -q scrapbook
!pip install -U -q "jupyter_server>=1.11.0"
!pip install -q fastparquet
!pip install -q import_from_github_com

     |████████████████████████████████| 335 kB 12.6 MB/s 
     |████████████████████████████████| 310 kB 43.1 MB/s 
     |████████████████████████████████| 197 kB 57.1 MB/s 
     |████████████████████████████████| 11.8 MB 32.7 MB/s 
     |████████████████████████████████| 4.2 MB 53.9 MB/s 
     |████████████████████████████████| 1.5 MB 38.4 MB/s 
     |████████████████████████████████| 61 kB 7.3 MB/s 
     |████████████████████████████████| 12.4 MB 28.2 MB/s 
     |████████████████████████████████| 636 kB 51.3 MB/s 
     |████████████████████████████████| 749 kB 54.6 MB/s 
     |████████████████████████████████| 10.9 MB 51.7 MB/s 
     |████████████████████████████████| 384 kB 58.0 MB/s 
     |████████████████████████████████| 6.6 MB 55.9 MB/s 
     |████████████████████████████████| 84 kB 2.6 MB/s 
     |████████████████████████████████| 130 kB 26.5 MB/s 
     |████████████████████████████████| 428 kB 58.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
path = '/content/gdrive/Shareddrives/현대홈쇼핑🛍/code/김범수/experiments/SASRec/0512_SASRec_FirstTry/'

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import re
import sys
import os
import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns',None)

from collections import defaultdict
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.datasets.amazon_reviews import get_review_data
from recommenders.datasets.split_utils import filter_k_core


# Transformer Based Models
from recommenders.models.sasrec.model import SASREC
from recommenders.models.sasrec.ssept import SSEPT

# Sampler for sequential prediction
from recommenders.models.sasrec.sampler import WarpSampler
from recommenders.models.sasrec.util import SASRecDataSet

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
Tensorflow version: 2.8.0


# 전처리 - make dataset for SASRec

In [ ]:
df = pd.read_parquet('/content/gdrive/Shareddrives/현대홈쇼핑🛍/data/raw/simple_preprocessing/0503_total_data.parquet',engine='fastparquet')

In [ ]:
df.head(1)

,고객번호,성별구분명,나이대,나이,가입연월,datetime,month,요일명,주문시간(시간),weekday-hour,주문번호,판매상품코드,기준상품코드,상품대분류명,상품중분류명,상품소분류명,상품세분류명,브랜드명,판매상품명,판매매체대분류명,판매매체중분류명,판매매체소분류명,판매매체명,live_or_web,GMV,할인금액,실제매출금액,순주문수량,price,PGM코드,PGM명,방송편성번호,쇼호스트,broadcast_datetime,recency,frequency,monetary,r,f,m,rfm,마케팅동의거부여부
0,200101002134,기타,40,45,200101,2021-05-11 18:00:00,5,화,18,화-18,20210511377616,2127664228,2100823100,식품,가공식품,수산가공식품,생선류,옥주부,옥주부 생선까스 새우까스 2종 20팩,모바일,eTV(M),방송상품 웹전시(M),일반매장에서 팔린 생방송(M),live,54454,2000,57900,1,54454,215764,옥주부생선까스(식품),202105113011,"하은지,이호준",2021-05-11 17:40:00,-1,0,0,5,5,5,15,N



일단 판매상품코드로 해보자

In [ ]:
df.rename({
  '고객번호':'userID',
  '판매상품코드':'itemID'
},axis=1,inplace=True)

In [ ]:
filtered_df = filter_k_core(df, 10,col_user="userID", col_item="itemID")

In [ ]:
import pickle
filtered_df.to_pickle('filtered_df.pkl')

In [ ]:
df['counter']=1
filtered_df['counter']=1
df_user_min = df[['userID','counter']].groupby('userID').sum()['counter'].min()
df_item_min = df[['itemID','counter']].groupby('itemID').sum()['counter'].min()
filtered_df_user_min = filtered_df[['userID','counter']].groupby('userID').sum()['counter'].min()
filtered_df_item_min = filtered_df[['itemID','counter']].groupby('itemID').sum()['counter'].min()

In [ ]:
print(
  'df_user_min', df_user_min,
  'df_item_min', df_item_min,
  'filtered_df_user_min', filtered_df_user_min,
  'filtered_df_item_min', filtered_df_item_min,
)

df_user_min 1 df_item_min 1 filtered_df_user_min 10 filtered_df_item_min 10


10회 이상 구매한 유저 + 10회 이상 구매된 상품만 남김

In [ ]:
print(
  df.shape,
  filtered_df.shape
)
# 그래도 300만 행은 남음!!

(13008967, 43) (3352443, 43)


In [ ]:
# user, item label encoding dictionary 생성
user_set, item_set = set(filtered_df['userID'].unique()), set(filtered_df['itemID'].unique())
user_map = dict()
item_map = dict()
for u, user in enumerate(user_set):
    user_map[user] = u+1
for i, item in enumerate(item_set):
    item_map[item] = i+1

In [ ]:
user_item_map = [user_map, item_map]
with open(path+'user_item_map.pkl','wb') as f:
  pickle.dump(user_item_map, f)

In [ ]:
filtered_df.rename({'datetime':'time'},inplace=True,axis=1)

In [ ]:
filtered_df["userID"] = filtered_df["userID"].apply(lambda x: user_map[x])
filtered_df["itemID"] = filtered_df["itemID"].apply(lambda x: item_map[x])
filtered_df = filtered_df.sort_values(by=["userID", "time"])
final_df = filtered_df[['userID','itemID']]
final_df.to_csv('sas_df.txt', sep="\t", header=False, index=False)

# Train the model

In [6]:
data = SASRecDataSet(filename=path+'sas_df.txt', col_sep="\t")

In [7]:
data.split()

In [8]:
num_epochs = 100
batch_size = 256
RANDOM_SEED = 100  # Set None for non-deterministic result

lr = 0.001             # learning rate
maxlen = 50            # maximum sequence length for each user
num_blocks = 6         # number of transformer blocks
hidden_units = 256     # number of units in the attention calculation
num_heads = 2          # number of attention heads
dropout_rate = 0.5     # dropout rate
l2_emb = 0.0           # L2 regularization coefficient
num_neg_test = data.itemnum-1     # number of negative examples per positive example -> for test and yielding metrics(nDCG@10/HR@10)
                         # num_neg_test=data.itemnum-1 -> yield metrics based on candidates including all items.
model_name = 'sasrec'  # 'sasrec' or 'ssept'

In [9]:
num_steps = int(len(data.user_train) / batch_size)
cc = 0.0
for u in data.user_train:
    cc += len(data.user_train[u])
print('%g Users and %g items' % (data.usernum, data.itemnum))
print('average sequence length: %.2f' % (cc / len(data.user_train)))
print('num_steps: ', num_steps)

187405 Users and 11416 items
average sequence length: 15.89
num_steps:  732


In [26]:
if model_name == 'sasrec':
    model = SASREC(item_num=data.itemnum,
                   seq_max_len=maxlen,
                   num_blocks=num_blocks,
                   embedding_dim=hidden_units,
                   attention_dim=hidden_units,
                   attention_num_heads=num_heads,
                   dropout_rate=dropout_rate,
                   conv_dims = [256, 256],
                   l2_reg=l2_emb,
                   num_neg_test=num_neg_test,
    )
elif model_name == "ssept":
    model = SSEPT(item_num=data.itemnum,
                  user_num=data.usernum,
                  seq_max_len=maxlen,
                  num_blocks=num_blocks,
                  # embedding_dim=hidden_units,  # optional
                  user_embedding_dim=10,
                  item_embedding_dim=hidden_units,
                  attention_dim=hidden_units,
                  attention_num_heads=num_heads,
                  dropout_rate=dropout_rate,
                  conv_dims = [110, 110],
                  l2_reg=l2_emb,
                  num_neg_test=num_neg_test
    )
else:
    print(f"Model-{model_name} not found")

In [27]:
sampler = WarpSampler(data.user_train, data.usernum, data.itemnum, batch_size=batch_size, maxlen=maxlen, n_workers=5)

# train 함수 재정의

In [12]:
def train(model, dataset, sampler, **kwargs):
        """
        High level function for model training as well as
        evaluation on the validation and test dataset
        
        <kwargs>
        num_epochs | batch_size | learning_rate | val_epoch : 몇 epoch마다 validation 진행할지
        """
        num_epochs = kwargs.get("num_epochs", 10)
        batch_size = kwargs.get("batch_size", 128)
        lr = kwargs.get("learning_rate", 0.001)
        val_epoch = kwargs.get("val_epoch", 5)

        num_steps = int(len(dataset.user_train) / batch_size)

        optimizer = tf.keras.optimizers.Adam(
            learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-7
        )

        loss_function = model.loss_function

        train_loss = tf.keras.metrics.Mean(name="train_loss")

        train_step_signature = [
            {
                "users": tf.TensorSpec(shape=(None, 1), dtype=tf.int64),
                "input_seq": tf.TensorSpec(
                    shape=(None, model.seq_max_len), dtype=tf.int64
                ),
                "positive": tf.TensorSpec(
                    shape=(None, model.seq_max_len), dtype=tf.int64
                ),
                "negative": tf.TensorSpec(
                    shape=(None, model.seq_max_len), dtype=tf.int64
                ),
            },
            tf.TensorSpec(shape=(None, 1), dtype=tf.int64),
        ]

        @tf.function(input_signature=train_step_signature)
        def train_step(inp, tar):
            with tf.GradientTape() as tape:
                pos_logits, neg_logits, loss_mask = model(inp, training=True)
                loss = loss_function(pos_logits, neg_logits, loss_mask)

            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            train_loss(loss)
            return loss

        T = 0.0
        t0 = Timer()
        t0.start()

        for epoch in range(1, num_epochs + 1):

            print(f'epoch {epoch} / {num_epochs} started---------------------')

            step_loss = []
            train_loss.reset_states()
            for step in tqdm(
                range(num_steps), total=num_steps, ncols=70, leave=False, unit="b"
            ):

                u, seq, pos, neg = sampler.next_batch()

                inputs, target = model.create_combined_dataset(u, seq, pos, neg)

                loss = train_step(inputs, target)
                step_loss.append(loss)

            if epoch % val_epoch == 0:                
                print("Evaluating...")
                t_test = evaluate(model,dataset)
                print(
                    f"epoch: {epoch}, time: {T},  test (NDCG@10: {t_test[0]}, HR@10: {t_test[1]})"
                )

# evaluate 함수 재정의

In [13]:
from tqdm import tqdm
import random
def evaluate(model_, dataset, target_user_n=1000, target_item_n=-1):
        """
        Evaluation on the test users (users with at least 3 items)

        <kwargs>
        model_ | dataset: SASRecDataSet 객체 | target_user_n: evaluate할 user 수 | target_item_n=
        """
        usernum = dataset.usernum
        itemnum = dataset.itemnum
        train = dataset.user_train  # removing deepcopy
        valid = dataset.user_valid
        test = dataset.user_test

        NDCG = 0.0
        HT = 0.0
        valid_user = 0.0

        if usernum > target_user_n:
            users = random.sample(range(1, usernum + 1), target_user_n)
        else:
            users = range(1, usernum + 1)
        # users = range(1,11)

        for u in tqdm(users, ncols=70, leave=False, unit="b"):

            if len(train[u]) < 1 or len(test[u]) < 1:
                continue

            seq = np.zeros([model_.seq_max_len], dtype=np.int32)
            idx = model_.seq_max_len - 1
            seq[idx] = valid[u][0]
            idx -= 1
            for i in reversed(train[u]):
                seq[idx] = i
                idx -= 1
                if idx == -1:
                    break
            rated = set(train[u])
            # print('rated', rated)
            rated.add(0)
            # print('rated2', rated)
            item_idx = [test[u][0]]
            # for _ in range(model_.num_neg_test):
            #     t = np.random.randint(1, itemnum + 1)
            #     while t in rated:
            #         t = np.random.randint(1, itemnum + 1)
            #     item_idx.append(t)
            '''
            item_num = list(range(1,11))
            rated = set([1,2,3,4])
            item_idx=[5]
            exclude_set = rated.union(set(item_idx))
            item_idx=item_idx+(list(set(item_num).difference(exclude_set)))
            '''
            exclude_set = rated.union(set(item_idx))
            item_idx=item_idx+list(set(range(1,itemnum+1)).difference(exclude_set))+[0 for _ in range(0,len(exclude_set)-2)]
            
            inputs = {}
            inputs["user"] = np.expand_dims(np.array([u]), axis=-1)
            inputs["input_seq"] = np.array([seq])
            inputs["candidate"] = np.array([item_idx])
            # print(inputs)

            # inverse to get descending sort
            predictions = -1.0 * model_.predict(inputs)
            predictions = np.array(predictions)
            predictions = predictions[0]
            # print('predictions:', predictions)

            rank = predictions.argsort().argsort()[0]
            # print('rank:', rank)


            valid_user += 1

            if rank < 10:
                NDCG += 1 / np.log2(rank + 2)
                HT += 1

        return NDCG / valid_user, HT / valid_user

In [49]:
train(model,data,sampler,num_epochs=1, batch_size=batch_size, learning_rate=lr, val_epoch=5)

epoch 1 / 1 started---------------------


# test save and load model

In [43]:
import pickle 
from recommenders.models.sasrec.model import SASREC
arg_list = ['item_num','seq_max_len','num_blocks','embedding_dim','attention_dim','attention_num_heads','dropout_rate','conv_dims','l2_reg','num_neg_test']
def save_sasrec_model(model,path, exp_name='sas_experiment'):
  model.save_weights(path+exp_name+'_weights') # save trained weights
  dict_to_save = {a: model.__dict__[a] for a in arg_list}
  with open(path+exp_name+'_model_args','wb') as f:
    pickle.dump(dict_to_save, f)

def load_sasrec_model(path, exp_name='sas_experiment'):
  with open(path+exp_name+'_model_args','rb') as f:
    arg_dict = pickle.load(f)
  model = SASREC(item_num=arg_dict['item_num'],
                   seq_max_len=arg_dict['seq_max_len'],
                   num_blocks=arg_dict['num_blocks'],
                   embedding_dim=arg_dict['embedding_dim'],
                   attention_dim=arg_dict['attention_dim'],
                   attention_num_heads=arg_dict['attention_num_heads'],
                   dropout_rate=arg_dict['dropout_rate'],
                   conv_dims = arg_dict['conv_dims'],
                   l2_reg=arg_dict['l2_reg'],
                   num_neg_test=arg_dict['num_neg_test'],
    )
  model.load_weights(path+exp_name+'_weights')
  return model

In [36]:
evaluate(model,data, target_user_n=100)

(0.021894546246997503, 0.05)

In [44]:
save_sasrec_model(model, path, exp_name='save_test')

In [45]:
loaded = load_sasrec_model(path, exp_name='save_test')

In [48]:
evaluate(loaded,data, target_user_n=100)

(0.02271110474028678, 0.05)

In [31]:
t = {'a':1,'b':2}
for i in t.items():
  print(i[0], i[1])

a 1
b 2


In [34]:
t['a']

1

In [15]:
model.save_weights(path+'saved_weight')

In [16]:
m= SASREC(item_num=data.itemnum,
                   seq_max_len=maxlen,
                   num_blocks=num_blocks,
                   embedding_dim=hidden_units,
                   attention_dim=hidden_units,
                   attention_num_heads=num_heads,
                   dropout_rate=dropout_rate,
                   conv_dims = [256, 256],
                   l2_reg=l2_emb,
                   num_neg_test=num_neg_test,
    )

In [17]:
m.load_weights(path+'saved_weight')

In [22]:
evaluate(m,data, target_user_n=100)

(0.02099267181450586, 0.05)

In [23]:
evaluate(model,data, target_user_n=100)

(0.0, 0.0)

In [ ]:
model.save('model')

In [ ]:
import pickle
with open(path+'model_final.pickle', 'wb') as f:
    pickle.dump(model, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(path+'model_final.pickle', 'rb') as f:
    m = pickle.load(f)

In [ ]:
from recommenders.utils.tf_utils import export_model
export_model(model,path)

TypeError: ignored

In [ ]:
m.update_kwargs()

AttributeError: ignored

In [ ]:
m(seq_max_len=50)

ValueError: ignored

In [ ]:
import os
from surprise import dump

dump.dump(path+'m_dumptest', algo=model)

In [ ]:
_, m = dump.load(path+'m_dumptest')

In [ ]:
evaluate(m,data)

AttributeError: ignored

In [ ]:
with Timer() as train_time:
    t_test = model.train(data, sampler, num_epochs=num_epochs, batch_size=batch_size, lr=lr, val_epoch=6)

print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))

Evaluating...


KeyboardInterrupt: ignored

In [ ]:
# save_model
tf.saved_model.save(model, path+'_ExpSetting')

In [ ]:
res_syn = {"ndcg@10": t_test[0], "Hit@10": t_test[1]}
print(res_syn)

{'ndcg@10': 0.07843458745948835, 'Hit@10': 0.1324}


In [ ]:
data.itemnum

11416

In [ ]:
item_idx

[5, 6, 7, 8, 9, 10]

In [ ]:
data.user_test[1]

[7546]

In [ ]:
data.user_valid[1]

[2764]

In [ ]:
data.user_train[1]

[3084, 11387, 5759, 10360, 10296, 5612, 7956, 5951, 1548, 3271, 1096, 7662]

In [ ]:
evaluate(m,data, target_user_n=1)

AttributeError: ignored

In [ ]:
# 1~100번 user predict
users_for_pred = range(1,101)
seq = np.zeros([model.seq_max_len], dtype=np.int32)
test = data.user_test
for u in users_for_pred:
  inputs = {}
  inputs["user"] = np.expand_dims(np.array([u]), axis=-1)
  inputs["input_seq"] = np.array([seq])
  inputs["candidate"] = np.array([item_idx])

KeyError: ignored

In [ ]:
data.user_test

{1: [7546],
 2: [619],
 3: [11020],
 4: [5255],
 5: [10892],
 6: [11262],
 7: [2682],
 8: [9125],
 9: [8468],
 10: [9658],
 11: [10405],
 12: [6019],
 13: [8845],
 14: [3131],
 15: [201],
 16: [3181],
 17: [1],
 18: [4740],
 19: [1],
 20: [9946],
 21: [9226],
 22: [1277],
 23: [2166],
 24: [289],
 25: [8726],
 26: [2363],
 27: [3537],
 28: [9789],
 29: [10032],
 30: [8726],
 31: [615],
 32: [4710],
 33: [6966],
 34: [2105],
 35: [8882],
 36: [5655],
 37: [11045],
 38: [10479],
 39: [2788],
 40: [10156],
 41: [6127],
 42: [259],
 43: [3067],
 44: [56],
 45: [6835],
 46: [8439],
 47: [3630],
 48: [3534],
 49: [9445],
 50: [2361],
 51: [9786],
 52: [9955],
 53: [10648],
 54: [2821],
 55: [9234],
 56: [4273],
 57: [8301],
 58: [3067],
 59: [10795],
 60: [995],
 61: [1096],
 62: [3091],
 63: [9763],
 64: [4478],
 65: [8968],
 66: [9726],
 67: [1319],
 68: [5754],
 69: [9718],
 70: [1281],
 71: [5731],
 72: [6694],
 73: [1327],
 74: [121],
 75: [272],
 76: [2337],
 77: [113],
 78: [9900],
 7

In [ ]:
model.predict()